# Criação da baseline

- Qual a métrica que iremos utilizar para avaliar qual será o melhor modelo?
    - Acurácia
    - Recall
    - Precision
    - F1-score

## 0. Setup

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

## 1. Carregando os dados

In [2]:
dados = pd.read_csv(filepath_or_buffer = '../data/raw/aug_train.csv')

dados.head(3)

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0


## 2. Manuseio dos dados

### 2.1. Exclusão da variável enrollee_id

In [3]:
dados = dados.drop(columns = 'enrollee_id', axis = 1)

dados.head(3)

,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0


In [4]:
#---- Selecionar as variáveis do banco de dados que são qualitativas

le = LabelEncoder()

list_colunas_qual = list(dados.select_dtypes(include = ['object']).columns)

dados[list_colunas_qual] = dados[list_colunas_qual].apply(lambda x: le.fit_transform(x.values))

In [5]:
dados.head(3)

,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,5,0.920,1,0,2,0,5,21,8,6,0,36,1.0
1,77,0.776,1,1,2,0,5,6,4,5,4,47,0.0
2,64,0.624,3,1,0,0,5,15,8,6,5,83,0.0


## 3. Criando a baseline

### 3.1. Divisão da base de treino e teste

In [6]:
X = dados.drop(columns = 'target', axis = 1)

y = dados.target

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, random_state = 19, stratify = y)

In [8]:
print(f'Quantidade de linhas do X_train: {X_train.shape[0]} \n \
Quantidade de linhas do X_test: {X_test.shape[0]}\n \
Quantidade de linhas do y_train: {y_train.shape[0]}\n \
Quantidade de linhas do y_test: {y_test.shape[0]}\
')

Quantidade de linhas do X_train: 13410 
 Quantidade de linhas do X_test: 5748
 Quantidade de linhas do y_train: 13410
 Quantidade de linhas do y_test: 5748


### 3.2. Escolha do modelo

In [11]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(random_state = 19, max_iter = 500)

In [12]:
lr.fit(X_train, y_train)

LogisticRegression(max_iter=500, random_state=19)

### 3.3. Predição do modelo nos dados de teste

In [18]:
y_pred = lr.predict(X_test)

# y_train: é o exemplo correto que o modelo viu para achar os padrões
# y_test: é o que vamos confrontar com o resultado do modelo
# y_pred: resultado do modelo

### 3.4. Validação do modelo

Etapa de confrontamento entre os resultados do modelo e o que realmente aconteceu

- **Casos predito 0 e vida real 0: (acerto)**
    - O modelo disse que a pessoa não está interessada em trocar de emprego e na vida real a pessoa não queria trocar de emprego
- **Casos predito 0 e vida real 1: (erro)**
    - O modelo disse que a pessoa não está interessada em trocar de emprego mas na vida real a pessoa queria trocar de emprego
- **Casos predito 1 e vida real 0: (erro)**
    - O modelo disse que a pessoa está interessada em trocar de emprego mas na vida real a pessoa não queria trocar de emprego
- **Casos predito 1 e vida real 1: (acerto)**
    - O modelo disse que a pessoa está interessada em trocar de emprego e na vida real a pessoa queria trocar de emprego

In [25]:
#---- Matriz de confusão

pd.crosstab(y_test, y_pred, rownames = ['Vida real'], colnames = ['Predito'], margins = True)

Predito,0.0,1.0,All
Vida real,,,
0.0,4040,275,4315
1.0,1063,370,1433
All,5103,645,5748


### 3.4.1. Interpretações:

- Nosso modelo acertou apenas **25%** das pessoas que realmente queriam trocar de trabalho (370/1433)
- Nosso modelo acertou 93% das pessoas que não queriam trocar de trabalho (4040/4315)
- Nosso modelo errou em 7% que as pessoas não queriam trocar de trabalho quando na verdade queriam (275/4315)
- Nosso modelo errou em 75% que as pessoas que queriam trocar de trabalho quando na verdade elas não queriam (1063/1433)

In [34]:
# Métricas a serem guardadas: Acurácia, Recall, Precision e F1-score

from sklearn.metrics import classification_report, accuracy_score, recall_score, f1_score, precision_score

print(classification_report(y_true = y_test, y_pred = y_pred))

              precision    recall  f1-score   support

         0.0       0.79      0.94      0.86      4315
         1.0       0.57      0.26      0.36      1433

    accuracy                           0.77      5748
   macro avg       0.68      0.60      0.61      5748
weighted avg       0.74      0.77      0.73      5748



In [35]:
accuracy_score(y_true = y_test, y_pred = y_pred) * 100

76.7223382045929

In [36]:
recall_score(y_true = y_test, y_pred = y_pred) * 100

25.819958129797627

In [38]:
precision_score(y_true = y_test, y_pred = y_pred) * 100

57.36434108527132

In [37]:
f1_score(y_true = y_test, y_pred = y_pred) * 100

35.6111645813282

## 4. Salvando o modelo

In [42]:
from joblib import dump, load

dump(value = lr, filename = '../models/model_baseline.joblib')

['../models/model_baseline.joblib']

## 5. Teste jogo rápido

In [44]:
modelo = load('../models/model_baseline.joblib')

modelo

LogisticRegression(max_iter=500, random_state=19)

In [50]:
modelo.predict_proba(X_test)

array([[0.86242928, 0.13757072],
       [0.87180193, 0.12819807],
       [0.89311914, 0.10688086],
       ...,
       [0.89178837, 0.10821163],
       [0.82650288, 0.17349712],
       [0.85911034, 0.14088966]])